### Metadata extraction

In [2]:
import pymupdf4llm
import pymupdf
from io import StringIO
from html.parser import HTMLParser
import re

class MLStripper(HTMLParser):
    def __init__(self):
        super().__init__()
        self.reset()
        self.strict = False
        self.convert_charrefs= True
        self.text = StringIO()
    def handle_data(self, d):
        self.text.write(d)
    def get_data(self):
        return self.text.getvalue()

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

def getItem(field_name, text):
    pattern = rf"{re.escape(field_name)}:*([^\r\n]+)"
    match = re.search(pattern, text)
    if match:
        report_no = match.group(1)
        return report_no.strip()
    return ""

def getItemWithStopWord(field_name, stop_word, text):

    pattern = rf"{re.escape(field_name)}:*([^\r\n]+?)({re.escape(stop_word)})"
    match = re.search(pattern, text)
    if match:
        report_no = match.group(1)
        return report_no.strip()

    return getItem(field_name, text)

import markdown

# Karotype testing
file = 'R24-0WH7-1.pdf'

# this is typical of reports coming at present from iGene
file = 'ORU_R01_R125.1_R0A.txt.pdf'

folder = 'Output/PDF/R01/'

with open(folder + file, 'rb') as g:
    pdf_stream = g.read()

    doc = pymupdf.open("pdf", pdf_stream)
    md_text = pymupdf4llm.to_markdown(doc)
    html_text = markdown.markdown(md_text)

clean_text = strip_tags(html_text)

print("Report No = "+getItem('Report No', clean_text))
print("Hospital No = "+getItem('Hospital No', clean_text))
print("NHS No = "+getItemWithStopWord('NHS No','Hive', clean_text))
print("Order Number (HIVE) = "+getItemWithStopWord('Hive Order ID','Path', clean_text))
print("Sample type = "+getItemWithStopWord('Sample','Sample', clean_text))
print("Sample2 type = "+getItem('Sample2', clean_text))
print("Pathology No. = "+getItem('Path. No', clean_text))
print("Specimen Collected = "+getItemWithStopWord('Collected','Activated', clean_text))
print("Specimen Collected2 = "+getItemWithStopWord('Collected2','Received2', clean_text))
print("Specimen Received2 = "+getItemWithStopWord('Received2','Received2', clean_text))

Report No = R25-00EK-3
Hospital No = Not provided
NHS No = :
Order Number (HIVE) = 1000152866
Sample type = Blood
Sample2 type = 
Pathology No. = ACC1810A1
Specimen Collected = Not provided
Specimen Collected2 = 
Specimen Received2 = 
